In [1]:
from datasets import load_dataset

data = load_dataset("milenamileentje/Dutch-Government-Data-for-Bias-detection")
print(data)
print(data["train"][0]) 
print(data["train"][0]["text"])
print(data["train"][0]["label"])


Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Column1', 'text', 'label'],
        num_rows: 1811
    })
    validation: Dataset({
        features: ['Column1', 'text', 'label'],
        num_rows: 713
    })
    test: Dataset({
        features: ['Column1', 'text', 'label'],
        num_rows: 782
    })
})
{'Column1': 8839, 'text': 'mens (crm) oordeelde dat een onderwijsstichting geen verboden onderscheid maakte door een meisje met down syndroom niet door te laten gaan op de reguliere school (oordeelnummer 2011–144).', 'label': 1.0}
mens (crm) oordeelde dat een onderwijsstichting geen verboden onderscheid maakte door een meisje met down syndroom niet door te laten gaan op de reguliere school (oordeelnummer 2011–144).
1.0


In [ ]:
def compute_metrics1(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    pred_labels = []
    for text in decoded_preds:
        match = re.search(r"\b(0|1)\b", text)
        pred_labels.append(int(match.group(1)) if match else -1)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    gold_labels = []
    for text in decoded_labels:
        match = re.search(r"\b(0|1)\b", text)
        gold_labels.append(int(match.group(1)) if match else -1)
    for i in range(len(gold_labels)):
        print(f"Predicted: {pred_labels[i]}, Gold: {gold_labels[i]}")   
    
    return accuracy.compute(predictions=pred_labels, references=gold_labels)

In [2]:
!pip install scikit-learn

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from datasets import load_dataset, Dataset

from transformers import Trainer, TrainingArguments
import evaluate
import numpy as np
import re
import torch
from transformers import Seq2SeqTrainer
import torch
import torch.nn.functional as F

In [49]:
model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

dataset = load_dataset("milenamileentje/Dutch-Government-Data-for-Bias-detection")

def preprocess(example, makeitwords=True):
    model_inputs = tokenizer(example['text'], truncation=True, padding="max_length", max_length=512)
    with tokenizer.as_target_tokenizer():
        if makeitwords:
            if example["label"] == 0:
                label = "niet-biased"
                labels = tokenizer(label, truncation=True, padding="max_length", max_length=512)
            elif example["label"] == 1: 
                label = "biased"
                labels = tokenizer(label, truncation=True, padding="max_length", max_length=512)
        else:
            labels = tokenizer(str(example["label"]), truncation=True, padding="max_length", max_length=512) 
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    return model_inputs

small_train_dataset = dataset["train"].select(range(1))
small_val_dataset = dataset["validation"].select(range(1))
tokenized_train = small_train_dataset.map(preprocess)
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val = small_val_dataset.map(preprocess)
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
accuracy = evaluate.load("accuracy")


training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,                     
    per_device_train_batch_size=1,         
    per_device_eval_batch_size=1,         
    num_train_epochs=1,                   
    weight_decay=0.0,                      
    save_strategy="no",                    
    logging_dir="./logs",
    logging_steps=1,                       
    report_to="none"
)
training_args.generation_config = None 
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer)


trainer.train()



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

C:\Users\inakl\AppData\Local\Temp\ipykernel_31952\1100552167.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
c:\Users\inakl\anaconda3\envs\atics_group\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,50.371300


TrainOutput(global_step=1, training_loss=50.371280670166016, metrics={'train_runtime': 15.111, 'train_samples_per_second': 0.066, 'train_steps_per_second': 0.066, 'total_flos': 1199046131712.0, 'train_loss': 50.371280670166016, 'epoch': 1.0})

In [50]:
print(tokenized_train)
outputs = model.generate(tokenized_train["input_ids"], attention_mask=tokenized_train["attention_mask"])    
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


Dataset({
    features: ['Column1', 'text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 1
})
['<extra_id_0>']
